In [1]:
import json, nltk, io, pickle
import numpy as np
from itertools import chain

from HRED_v2 import HRED

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 780 (CNMeM is enabled with initial size: 30.0% of memory, cuDNN 5105)


In [2]:
data = np.load('/pio/data/data/squad/train.pkl')
w_to_i = {}
idx = 0

with io.open('/pio/data/data/squad/wordlist.txt', 'r', encoding='utf-8') as f:
    for line in f:
        w_to_i[line[:-1]] = idx
        idx += 1
        
i_to_w = {v:k for (k,v) in w_to_i.items()}

w_to_i['<pad_value>'] = -1
i_to_w[-1] = '<pad_value>'

voc_size = len(w_to_i)

def get_w(idx):
    return i_to_w[idx]

In [3]:
def trim_data(data, trim=100):
    return [d for d in data if max(map(len, d[1])) <= trim]

data_trimmed = trim_data(data)
len(data), len(data_trimmed)

(87599, 86789)

In [4]:
hred_net = HRED(voc_size=voc_size,
                emb_size=100,
                lv1_rec_size=300, 
                lv2_rec_size=300, 
                out_emb_size=100)

Building the model...
Compiling theano functions...
Done


In [5]:
ans = hred_net.get_output(data[:12], 3)
ans[0]

[array([ 0.54303432,  0.53078395,  0.499749  ,  0.51067436,  0.51147985,
         0.50409448,  0.49557057,  0.49993944,  0.49953365], dtype=float32),
 array([ 0.51827729,  0.49949208,  0.51738709,  0.50401407,  0.50157839,
         0.50367337,  0.51299101,  0.50543135,  0.50714558,  0.51372296,
         0.51686668,  0.51427102,  0.48752847,  0.47258207,  0.46597269,
         0.47980109], dtype=float32),
 array([ 0.505162  ,  0.48032287,  0.49906838,  0.49898484,  0.47359842,
         0.49529317,  0.48794621,  0.49679899,  0.49610531,  0.50689596,
         0.50199521,  0.50135016,  0.50532329,  0.5103032 ,  0.51402503,
         0.51320779,  0.51263028,  0.51967037,  0.51594871,  0.51989752,
         0.52553964,  0.49501914,  0.50797409,  0.50624341,  0.51008105,
         0.5125246 ,  0.51760632,  0.51998937,  0.51627296,  0.51194686], dtype=float32),
 array([ 0.50041193,  0.47737885,  0.46319419,  0.48196873,  0.47749609,
         0.48813593,  0.46737525,  0.48806357,  0.48631111,  0.46

In [ ]:
list(chain(*ans[5]))[46]

In [ ]:
get_w(list(chain(*data[5][1][1:]))[45])

In [ ]:
list(reversed(np.array(list(chain(*ans[8]))).argsort()))

In [ ]:
# Zwykły trening nie robił nic: wartości zwracane przez sieć dla zbioru uczącego nie miały żadnego sensu.
# Największe prawdopodobieństwa były prawie niezależne od pytania, miało ono mały wpływ na to, co się dzieje.
# Dla kosztu weighted_bin_ce mocno skrzywionej w stronę sytuacji t=1 wyszło podobnie.
#
# Następny krok: użyć zanurzeń GLOVE, dodać cechy z https://arxiv.org/abs/1703.04816

In [11]:
np.arange(30)[slice(0,28)]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27])

In [6]:
hred_net.train_one_epoch(data_trimmed, 5, log_interval=20) # weighted cost

Done 20 batches in 7.96s	training loss:	0.813393
Done 40 batches in 16.89s	training loss:	0.618499
Done 60 batches in 25.51s	training loss:	0.537964
Done 80 batches in 33.72s	training loss:	0.476587
Done 100 batches in 41.79s	training loss:	0.436442
Done 120 batches in 48.56s	training loss:	0.419979
Done 140 batches in 57.10s	training loss:	0.409618
Done 160 batches in 65.07s	training loss:	0.408088
Done 180 batches in 74.05s	training loss:	0.397444
Done 200 batches in 81.68s	training loss:	0.401948
Done 220 batches in 90.27s	training loss:	0.401396
Done 240 batches in 98.95s	training loss:	0.410639
Done 260 batches in 107.43s	training loss:	0.418680
Done 280 batches in 115.18s	training loss:	0.427639
Done 300 batches in 123.33s	training loss:	0.426674
Done 320 batches in 131.00s	training loss:	0.422292
Done 340 batches in 139.84s	training loss:	0.424456
Done 360 batches in 148.91s	training loss:	0.415105
Done 380 batches in 156.67s	training loss:	0.416838
Done 400 batches in 163.90s	t

KeyboardInterrupt: 

In [5]:
hred_net.train_one_epoch(data_trimmed, 5, log_interval=20)

Done 20 batches in 2.25s	training loss:	0.228349
Done 40 batches in 5.03s	training loss:	0.154742
Done 60 batches in 7.65s	training loss:	0.126147
Done 80 batches in 10.05s	training loss:	0.106807
Done 100 batches in 12.52s	training loss:	0.095184
Done 120 batches in 14.64s	training loss:	0.090502
Done 140 batches in 17.25s	training loss:	0.087078
Done 160 batches in 19.51s	training loss:	0.085557
Done 180 batches in 22.31s	training loss:	0.082334
Done 200 batches in 24.40s	training loss:	0.082256
Done 220 batches in 26.96s	training loss:	0.081215
Done 240 batches in 29.60s	training loss:	0.082376
Done 260 batches in 32.51s	training loss:	0.083784
Done 280 batches in 34.97s	training loss:	0.084956
Done 300 batches in 37.36s	training loss:	0.084330
Done 320 batches in 39.52s	training loss:	0.083391
Done 340 batches in 42.24s	training loss:	0.083563
Done 360 batches in 44.99s	training loss:	0.081207
Done 380 batches in 47.14s	training loss:	0.081512
Done 400 batches in 49.09s	training lo

KeyboardInterrupt: 

In [ ]:
hred_net.save_params()